# Lab - Choice Modeling

---
## *Don't forget to make a copy first!*
---

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf

from itertools import product
from seaborn import plt

# Example 1:

Adapted from http://www.ats.ucla.edu/stat/r/dae/logit.htm

In [ ]:
x = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")

In [ ]:
x.describe().T

*...what does* `.T` *do?*

In [ ]:
x.pivot_table(values='gre', index='admit', columns='rank', aggfunc='count')  # cross-tab

## Linear Fit:

In [ ]:
lin = smf.ols('admit ~ gre + gpa + rank', x)
lin.fit().summary()

In [ ]:
lin0 = smf.ols('admit ~ 0 + gre + gpa + rank', x)
lin0.fit().summary()

## Logit:

In [ ]:
lgt = smf.logit('admit ~ gre + gpa + rank', x)
logit_fit = lgt.fit()
logit_fit.summary()

coeffs -> chg in log-odds of the output variable for unit increase in the input variable

### Factor Rank:

In [ ]:
x['rank_factor'] = x['rank'].astype(object)

##### *N.B.* `x.rank` *won't work here. We have to use* `x['rank']`

In [ ]:
lgt = smf.logit('admit ~ gre + gpa + rank_factor', x)
logit_fit = lgt.fit()
logit_fit.summary()

*...why factor?*

coeffs of indicator (dummy) vars are slightly different...for example, the coeff of rank2 represents the change in the log-odds of the output variable that comes going to a rank2 school instead of a rank1 school

odds ratios can be found by exponentiating the log-odds ratios:

In [ ]:
logit_fit.params.map(np.exp)

#### *N.B. important to give columns the same names as in the model:*

In [ ]:
new_data = pd.DataFrame({'gre': x.gre.mean(), 'gpa': x.gpa.mean(), 
                         'rank_factor': np.array(range(1,5), dtype=object)})
new_data

#### predict probs for new data (varying rank):

In [ ]:
new_data['rank_prob'] = logit_fit.predict(new_data)
new_data

#### predict probs for new data (varying gre):

In [ ]:
new_data2 = pd.DataFrame(list(product(np.linspace(200, 800, 100), 
                                      np.array(range(1,5), dtype=object))),
                         columns = ['gre', 'rank_factor'])
new_data2['gpa'] = x.gpa.mean()
new_data2.describe()

In [ ]:
new_data2['pred'] = logit_fit.predict(new_data2)

new_data2.pivot(index='gre', columns='rank_factor', values='pred').plot()

#### predict probs for new data (varying gpa):

In [ ]:
new_data3 = pd.DataFrame(list(product(np.linspace(0, 4, 100), 
                                      np.array(range(1,5), dtype=object))),
                         columns = ['gpa', 'rank_factor'])
new_data3['gre'] = x.gre.mean()
new_data3.describe()

In [ ]:
new_data3['pred'] = logit_fit.predict(new_data3)

new_data3.pivot(index='gpa', columns='rank_factor', values='pred').plot()

# Example 2:

*Courtesy of Nir Kaldero*

##### Data - Grocery type of data about Milk Consumption 

###### Variables:

###### id - unique number for each consumer, 500 observations 

###### product - binary variable (1,0); if product ==1 : consumer bought 2% milk, otherwise : fat-milk

###### full_price - full price before promotion (if any)

###### full_pri - the price after the discount/promotion 

###### disc_price - totall amount of discount

###### bundel - if consumers buy the products as a bundel (2 per 6, 1 per 3)

###### time_day : 1== morning (until noon), otherwise: after noon-close

###### repeated? - if consumer i is a repated buyer in the store

###### repeated_bundel? - if consumer already buy the product as a bundel before

## Pull the data

In [ ]:
mdata = pd.read_csv('data/milkdata.csv')
mdata.head(3)

## summary statistics (all variables)

In [ ]:
mdata.describe().T

## plot the distribution (or  density) of full_pri (price after promotion)

In [ ]:
mdata.full_pri.hist()

## plot the distribution of promo (total promotion)

In [ ]:
mdata.disc_price.hist()

## Run a simple logit model where yi = Prob(product i = 1) on all other variables in the data

In [ ]:
mdata.columns[2:]

In [ ]:
train_cols = mdata.columns[2:]

In [ ]:
logit = smf.Logit(mdata["product"], mdata[train_cols])

In [ ]:
results=logit.fit()
results.summary()

### Questions:

## What we are tryng to find?

### 1. What is the expeced probability that a consumer will buy 2% milk if all other variables are equal to the avegrage (mean) number in the whole sample?

### 2. Which variables are signficant and which are not? (95 percent confident)

### 3. Which variables are consistent with your prior intuition and which are not?

### 4. By reading the output from this regression - would you recommend for the Marketing Team to sell milk in bundle? yes? no? explain?

## Interpret the estimated (betas):

## Pair Excercise:

### 1. Run the same model with LPM

### 2. Predict y_hat

### 3. Plot the distribution of y_hat, is there a problem?

### 4. Plot the distribution of y_hat from the logit model, is there a problem?

## On your own:

### 1. Run the next logit model:

#### y = bundel (if buyera are buying milk in bundels)

#### x = product, full_price, full_pri, promo, disc_pricem, time_day, repeated

### 2. Are consumer more likley to buy 2% milk vs. fat-milk? yes or no? explain

### 3. Is the effect of promotion negative or positive on the outcome (Ignore significance)? Can promotions drive consumer to buy in boundle?

### 4. Calculate the odds ratio for this regression

### 5. Can you think, with the results we got from this regression, about a strategy to convert consumers to buy halthier milk (2%) rather than fat-milk? 

---

# Don't forget to fill out: http://tinyurl.com/dat-exit-ticket !